In [4]:
using MAT, Statistics, LinearAlgebra,Random, Plots

In [5]:
include("mvnrnd.jl")

mvnrnd (generic function with 1 method)

Reads in all the variables optained that are used in line 93 and onwards and are defined before that.

In [6]:
function string_as_varname(s::AbstractString,v::Any)
         s=Symbol(s)
         @eval (($s) = ($v))
end

file = matopen("simulationData.mat")

#Reads in ju[IO, price, Omega, data, labor, mu, aggtfp, alpha, grossy, stfp, beta, agggdp, Sigma, capital, vadd]

varnames = names(file)
for varname in varnames
    val = read(file, varname)
    # Instantiate the variable with its name in the current scope
    string_as_varname(varname,val)
    end

close(file)

We now want to figure out what every variable stands for, and where it comes from.
So far we know:
- **IO**: Input-Output Matrix
- **STFP** Sectoral TFP Groth per anno
- **$\Omega$**: I think this is the same as in *Definition 3* Es gilt $$(\Omega)_i^j = \frac {(\mathit{IO})_i^j} {\sum_{k=0}^N(\mathit{IO}_k^j)}$$ Also der Anteil eines Sektors $i$ an der Gesamptproduktion des Sektors $j$
- **$\Sigma$**: Covarianz von SFTP
- **$\mu$**: Rowwise mean of stfp. So the mean annual groth of each sector
- **data**: quantity data matrix
- **price**: price data matrix. I think price and data are for our anyalsis quite useless, they are used as building blocks

This is not used anywhere in the code.

In [7]:
cum_stfp = cumsum(log.(1 .+ stfp)',dims=1);
cum_stfp_4year = cum_stfp[:,begin:4:end];
covariance_4year = Statistics.cov(cum_stfp_4year,dims=2,corrected=false);

Here comes the randomness. As far as i can tell the generated vectors have mean $\mu = \text{Var}(\mathit{stfp})$ and $\sigma = \text{Var}(\mathit{stfp})$ Because $\sigma$ is a Diagonal Matrix, there is no correlation between each random number, what makes this rather perculiar. Intuition wise I would say, that this implies, that the shocks are independent of one-another.

In [8]:
A = exp.(mvnrnd(-1/2*diag(Sigma),diagm(diag(Sigma))));

In [9]:
price

4048×94 Matrix{Float64}:
 1960.0   1.0  0.33   0.211  0.081  …  1.0  1.0  1.0  0.161  1.0  0.207
 1961.0   1.0  0.337  0.229  0.087     1.0  1.0  1.0  0.166  1.0  0.201
 1962.0   1.0  0.352  0.227  0.095     1.0  1.0  1.0  0.164  1.0  0.196
 1963.0   1.0  0.349  0.223  0.101     1.0  1.0  1.0  0.17   1.0  0.191
 1964.0   1.0  0.347  0.182  0.121     1.0  1.0  1.0  0.171  1.0  0.214
 1965.0   1.0  0.364  0.225  0.126  …  1.0  1.0  1.0  0.174  1.0  0.213
 1966.0   1.0  0.4    0.249  0.142     1.0  1.0  1.0  0.179  1.0  0.222
 1967.0   1.0  0.379  0.224  0.157     1.0  1.0  1.0  0.183  1.0  0.213
 1968.0   1.0  0.385  0.212  0.173     1.0  1.0  1.0  0.191  1.0  0.209
 1969.0   1.0  0.432  0.306  0.185     1.0  1.0  1.0  0.2    1.0  0.218
 1970.0   1.0  0.44   0.309  0.189  …  1.0  1.0  1.0  0.211  1.0  0.235
 1971.0   1.0  0.429  0.365  0.178     1.0  1.0  1.0  0.226  1.0  0.235
 1972.0   1.0  0.521  0.514  0.183     1.0  1.0  1.0  0.233  1.0  0.256
    ⋮                               ⋱  